In [2]:
import pandas as pd
import csv
import pickle
import numpy as np
from IPython.display import display
import multiprocessing
from multiprocessing import Pool
import math
from tqdm import tqdm
import time
import traceback
import json
import numpy as np

In [4]:
#path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"
path = '/Users/syamantak/JayateeB/new_files/data/'
event = 'nyc'
interval = 30
current_time = 420
start_hour = 7

In [3]:
def load_pickle_file(pickled_file):
    print(f'Loading data file from {pickled_file}')
    infile = open(pickled_file,'rb')
    unpickled_file = pickle.load(infile)
    print(f'Loaded {len(unpickled_file)} entries')
    infile.close()
    return unpickled_file
          
    
def save_pickle_file(path, data):
    print('Dumping data to path {}'.format(path))
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    print('Finished dumping data to path {}'.format(path))


def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y



In [6]:
users

,text,time_lapsed,order,created_at,description,favourites_count,followers_count,followers_json,friends_count,friends_json,...,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count,source_ids
0,RT @tomwatson: Timely coverage of this brutal ...,0.00,1.0,2016-01-13 03:30:20,Using my anger and humor to make our country b...,53462,4850,"[2563612535, 1967838145, 781382047319232512, 1...",4869,"[259527297, 45214107, 2742608846, 2756668840, ...",...,0,0,0,699,61.303290,6.938484,76.483548,0.017167,6.965665,"[49115780, 407396556, 950531, 50011708, 258885..."
1,NYC Pedestrian Safety Comes Under Scrutiny Aft...,8.90,2.0,2017-07-13 19:40:39,Who needs silly stoplights and parking tickets...,0,881,[],771,[],...,1,0,0,151,218.894040,5.834437,0.000000,0.039735,5.105960,[]
2,RT @NYGovCuomo: Terror won’t beat New York bec...,9.95,3.0,2015-12-06 19:35:08,100% Trump,19623,5468,"[953288658387460096, 788717662088159236, 94985...",5443,"[970693423, 994248527281770497, 186506052, 485...",...,2,0,0,736,110.214674,7.429348,26.661685,0.207880,7.395380,"[407396556, 3685247237, 760622447351599104, 19..."
3,RT @krassenstein: Trump is systematically tryi...,14.88,4.0,2009-10-22 20:26:38,,5773,179,"[1867596428, 503129500, 53577731, 2855110831, ...",457,"[875438321475870720, 981587930467782656, 98204...",...,3,0,0,2972,1.962315,0.060229,1.942463,0.000000,0.153769,"[2836421, 755835576, 132339474, 236487888, 160..."
4,RT @tomwatson: Timely coverage of this brutal ...,15.73,5.0,2017-10-09 10:40:00,,1652,35,"[1006369943519100929, 908087902155956225, 1011...",272,"[2839444615, 3161681103, 38936142, 510464011, ...",...,4,0,0,64,61.703125,0.546875,25.812500,0.000000,4.250000,"[14412533, 14173315, 2836421, 37590426, 815929..."
5,MANHATTAN *EXPLOSION* IN TRANSIT 8TH AV &amp; ...,19.50,6.0,2017-07-20 01:31:52,The most interesting news brought to you the f...,0,1116,"[3350973934, 1005543350462754816, 933627841509...",3451,"[37291805, 26329195, 78525538, 17446621, 88755...",...,5,0,0,145,0.682759,7.696552,0.000000,0.006897,23.800000,"[8862532, 17393196, 9229672, 17469289, 3825049..."
6,BREAKING Large explosion at Port Authority bus...,20.72,7.0,2010-07-31 22:52:40,Breaking News Guy (real Breaking News) Award-w...,140,1335,"[2821309229, 761111034140635136, 226433020, 14...",2059,"[516081403, 88749392, 177737764, 3685916295, 1...",...,6,0,0,2690,3.015242,0.496283,0.052045,0.026022,0.765428,"[125408014, 27840350, 16371691, 311010319, 828..."
7,RT @gattotony: BREAKING Large explosion at Por...,23.48,8.0,2010-03-22 18:21:52,Rabid Emmy winning reporter/anchor WPIX-TV--of...,5554,8616,"[3678807743, 1007572938655031298, 880936686930...",3298,"[2321010157, 2214315516, 20262083, 251918778, ...",...,6,1,2.76,2821,4.790854,3.054236,1.968805,0.066643,1.169089,"[173260818, 16371691, 311010319, 21751864, 886..."
8,Explosion at Port Authority Station https://t...,23.77,9.0,2009-09-30 06:15:50,Global Head of User Experience & Product Desig...,2588,382,"[2349574291, 778837981620678656, 23978832, 118...",192,"[7563792, 16425462, 803263807489536000, 149475...",...,8,0,0,2995,2.605342,0.127546,0.864107,0.006344,0.064107,"[828715817504219136, 17393196, 22637974, 61734..."
9,RT @gattotony: BREAKING Large explosion at Por...,24.90,10.0,2009-03-31 09:02:28,TV Journalist & Optimist | IG:donchampion RT's...,11337,5174,"[116028050, 3280889948, 17891808, 101970477393...",2774,"[309032583, 47632905, 305217958, 29769998, 220...",...,6,1,4.18,3178,10.455632,1.628068,3.567338,0.062618,0.872876,"[173260818, 22012474, 353869240, 27512619, 173..."


In [23]:
unique_users = set(users['id'])

In [24]:
def id_exists(df, uid):
    if uid in unique_users:
        return True
    return False

def filter_id_list(df, value):
        return list(filter(lambda uid: id_exists(df, uid), value))

In [25]:
users["source_ids"] = users["source_ids"].map(lambda x: filter_id_list(users, x))

In [32]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88266 entries, 0 to 88265
Data columns (total 31 columns):
text                           88266 non-null object
time_lapsed                    88266 non-null float64
order                          88266 non-null float64
created_at                     88266 non-null datetime64[ns]
description                    88266 non-null object
favourites_count               88266 non-null int64
followers_count                88266 non-null int64
followers_json                 88266 non-null object
friends_count                  88266 non-null int64
friends_json                   88266 non-null object
geo_enabled                    88266 non-null int64
id                             88266 non-null int64
lang                           88266 non-null object
listed_count                   88266 non-null int64
location                       88266 non-null object
name                           88266 non-null object
screen_name                    88266 no

In [29]:
if event == 'givenchy':
    followers = pd.read_csv(path+"mr.out",sep='\t',header = None)
elif event == 'nyc':
    followers = pd.read_csv(path+"nyc_users_6_9_followers.tsv",sep='\t',header = None)
    

In [8]:
ext_followers = pd.read_csv(path+'nyc_6_9_ext_followers.csv')

In [15]:
ext_followers['normalized_statuses_count'] = ext_followers['statuses_count'] / ext_followers['user_created_days']
ext_followers['normalized_followers_count'] = ext_followers['followers_count'] / ext_followers['user_created_days']
ext_followers['normalized_favourites_count'] = ext_followers['favourites_count'] / ext_followers['user_created_days']
ext_followers['normalized_listed_count'] = ext_followers['listed_count'] / ext_followers['user_created_days']
ext_followers['normalized_friends_count'] = ext_followers['friends_count'] / ext_followers['user_created_days']

In [11]:
import pickle

import pandas as pd


class SrcCandidateFilter(object):

    def __init__(self, users_file):
        self.users_file = users_file

    @staticmethod
    def load_pickle_file(pickled_file):
        print(f'Loading data file from {pickled_file}')
        infile = open(pickled_file, 'rb')
        unpickled_file = pickle.load(infile)
        print(f'Loaded {len(unpickled_file)} entries')
        infile.close()
        return unpickled_file

    @staticmethod
    def save_pickle_file(path, data):
        print('Dumping data to path {}'.format(path))
        with open(path, 'wb') as file:
            pickle.dump(data, file)
        print('Finished dumping data to path {}'.format(path))

    def filter_indices(self, candidate_idx_list, idx_out_of_range_set):
        if len(candidate_idx_list) > 0:
            return list(set(candidate_idx_list).difference(idx_out_of_range_set))
        return list()

    def index_to_id(self, df, idx):
        if idx:
            return int(df.loc[int(idx), 'id'])
        return None

    def prepare(self, followers_file):
        users_df = self.load_pickle_file(self.users_file)
        index_list = users_df[users_df['time_lapsed'] > 540].index.tolist()

        idx_out_of_range_set = set(index_list)
        users_df["source_candidates"] = users_df["source_candidates"].map(
            lambda x: self.filter_indices(x, idx_out_of_range_set))
        users_df = users_df[users_df['time_lapsed'] <= 540]
        followers = pd.read_csv(followers_file,
                                sep='\t',
                                header=None,
                                names=["id", "followers_list"])

        users_df = users_df.join(followers.set_index('id'),
                                 on='id',
                                 how="left")

        self.save_pickle_file(path + "nyc_users_6_9_infected.dat", users_df)

    def merge(self, ext_followers_df):
        pass



In [12]:
m = SrcCandidateFilter("/Users/syamantak/JayateeB/new_files/data/nyc/nyc_users.dat")
m.prepare("/Users/syamantak/JayateeB/dissertation/output/nyc_users_6_9_followers.tsv")


Loading data file from /Users/syamantak/JayateeB/new_files/data/nyc/nyc_users.dat
Loaded 135912 entries
Dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_users_6_9_infected.dat
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_users_6_9_infected.dat
88265    885801856848596993
Name: id, dtype: int64


In [13]:
infected_users_6_9 = load_pickle_file(path+'nyc_users_6_9_infected.dat')

Loading data file from /Users/syamantak/JayateeB/new_files/data/nyc_users_6_9_infected.dat
Loaded 88266 entries


In [45]:
ext_followers = pd.read_csv(path+'nyc_6_7_ext_followers.csv')

In [33]:
def process_data_idx(start_index, end_index, current_time):

    features = {
        #Columns which are added for simulation, but they are not used as features for model prediction
        'user_id':[],
        'infected_status':[],
        'infection_time':[],
        'followers_list':[],

        #Columns used as features for model prediction
        'UsM_deltaDays': [],
        'UsM_statusesCount': [],
        'UsM_followersCount': [],
        'UsM_favouritesCount': [],
        'UsM_friendsCount': [],
        'UsM_listedCount': [],
        'UsM_normalizedUserStatusesCount': [],
        'UsM_normalizedUserFollowersCount': [],
        'UsM_normalizedUserFavouritesCount': [],
        'UsM_normalizedUserListedCount': [],
        'UsM_normalizedUserFriendsCount': [],
        'UsM_deltaDays0': [],
        'UsM_statusesCount0': [],
        'UsM_followersCount0': [],
        'UsM_favouritesCount0': [],
        'UsM_friendsCount0': [],
        'UsM_listedCount0': [],
        'UsM_normalizedUserStatusesCount0': [],
        'UsM_normalizedUserFollowersCount0': [],
        'UsM_normalizedUserFavouritesCount0': [],
        'UsM_normalizedUserListedCount0': [],
        'UsM_normalizedUserFriendsCount0': [],
        'UsM_deltaDays-1': [],
        'UsM_statusesCount-1': [],
        'UsM_followersCount-1': [],
        'UsM_favouritesCount-1': [],
        'UsM_friendsCount-1': [],
        'UsM_listedCount-1': [],
        'UsM_normalizedUserStatusesCount-1': [],
        'UsM_normalizedUserFollowersCount-1': [],
        'UsM_normalizedUserFavouritesCount-1': [],
        'UsM_normalizedUserListedCount-1': [],
        'UsM_normalizedUserFriendsCount-1': [],
        # TwM: Tweet metadata
        'TwM_t0': [],
        'TwM_tSeed0': [],
        'TwM_t-1': [],
        'TwM_tSeed-1': [],
        'TwM_tCurrent': [],
        # Nw: Network
        'Nw_degree': [],
        'Nw_inDegree': [],
        'Nw_outDegree': [],
        'Nw_degree0': [],
        'Nw_inDegree0': [],
        'Nw_outDegree0': [],
        'Nw_degree-1': [],
        'Nw_inDegree-1': [],
        'Nw_outDegree-1': [],
        'Nw_degreeSeed0': [],
        'Nw_inDegreeSeed0': [],
        'Nw_outDegreeSeed0': [],
        'Nw_degreeSeed-1': [],
        'Nw_inDegreeSeed-1': [],
        'Nw_outDegreeSeed-1': [],
        # SNw: Spreading Network
        'SNw_nFriendsInfected': [],
        'SNw_friendsInfectedRatio': [],
        'SNw_generation0': [],
        'SNw_generation-1': [],
        'SNw_timeSinceSeed0': [],
        'SNw_timeSinceSeed-1': [],
        'SNw_totalNodesInfected': [],
        'SNw_nodeInfectedCentrality': [],
        'SNw_totalInDegree': [],
        'SNw_totalOutDegree': [],
        'SNw_inDegreeCentrality': [],
        'SNw_inDegreeCentrality0': [],
        'SNw_inDegreeCentrality-1': [],
        'SNw_outDegreeCentrality': [],
        'SNw_outDegreeCentrality0': [],
        'SNw_outDegreeCentrality-1': [],
        'SNw_inDegreeCentralitySeed0':[],
        'SNw_outDegreeCentralitySeed0':[],
        'SNw_inDegreeCentralitySeed-1':[],
        'SNw_outDegreeCentralitySeed-1':[],
        # Stat: Statistical
        'Stat_average_kOut': [],
        'Stat_average_t': [],
        'Stat_average_deltaDays': [],
        'Stat_average_statusesCount': [],
        'Stat_average_followersCount': [],
        'Stat_average_favouritesCount': [],
        'Stat_average_friendsCount': [],
        'Stat_average_listedCount': [],
        'Stat_average_normalizedUserStatusesCount': [],
        'Stat_average_normalizedUserFollowersCount': [],
        'Stat_average_normalizedUserFavouritesCount': [],
        'Stat_average_normalizedUserListedCount': [],
        'Stat_average_normalizedUserFriendsCount': [],
        'Stat_max_kOut': [],
        'Stat_min_kOut': []

    }

    with tqdm(total=len(network_simulation[start_index: end_index])) as pbar: 
        for index, user_row in network_simulation[start_index: end_index].iterrows():
            
            source_candidates = sorted(user_row['source_candidates']) if isinstance(user_row['source_candidates'], list) else np.nan
            features['user_id'].append(user_row['id'])
            features['infected_status'].append(False)
            features['infection_time'].append(None)
            #print(f"user_row['followers_list']:{user_row['followers_list']}")
            features['followers_list'].append(user_row['followers_list'])
            #print("b")
            features['UsM_deltaDays'].append(user_row['user_created_days'])
            features['UsM_statusesCount'].append(user_row['statuses_count'])
            features['UsM_followersCount'].append(user_row['followers_count'])
            features['UsM_favouritesCount'].append(user_row['favourites_count'])
            features['UsM_friendsCount'].append(user_row['friends_count'])
            features['UsM_listedCount'].append(user_row['listed_count'])
            features['UsM_normalizedUserStatusesCount'].append(user_row['normalized_statuses_count'])
            features['UsM_normalizedUserFollowersCount'].append(user_row['normalized_followers_count'])
            features['UsM_normalizedUserFavouritesCount'].append(user_row['normalized_favourites_count'])
            features['UsM_normalizedUserListedCount'].append(user_row['normalized_listed_count'])
            features['UsM_normalizedUserFriendsCount'].append(user_row['normalized_friends_count'])
            if isinstance(source_candidates, list):
                sources = network_simulation.loc[source_candidates]
                sources_dataframe = sources[sources['time_lapsed'] <= current_time]
                sources = sources_dataframe.index.tolist()
            else:
                sources = []
            #sources = [x for x in source_candidates if users.loc[x,'time_lapsed'] <= current_time]
            #print(f'sources:{sources}')
            if len(sources) > 0:

                # Assign the values here to save computation
                first_source_index = source_candidates[0]
                first_source_row = users.loc[first_source_index]
                first_source_seed_row = users.loc[first_source_row['seed_index']]

                inDegreeList = sources_dataframe.friends_count.tolist()
                outDegreeList = sources_dataframe.followers_count.tolist()
                degreeList = [x + y for x, y in zip(inDegreeList, outDegreeList)]
                
                s_ind = sources_dataframe.friends_count
                s_outd = sources_dataframe.followers_count
                outDegreeList = s_outd.tolist()
                inDegreeList = s_ind.tolist()
                degreeList = (s_ind + s_outd).tolist()

                #degreeList = list(users.loc[i, 'followers_count'] + users.loc[i, 'friends_count']  for i in sources)
                current_time_series = pd.Series([current_time] * len(sources))
                time_lapsed_series = sources_dataframe.time_lapsed
                timeList = (current_time_series - time_lapsed_series).tolist()


                last_source_index = sources[-1]
                last_source_row = network_simulation.loc[last_source_index]
                last_source_seed_row = network_simulation.loc[last_source_row['seed_index']]

                usr_index = index

                if user_row['time_lapsed'] <= current_time:

                    features['infected_status'][-1] = True
                    features['infection_time'][-1] = user_row['time_lapsed']

                    network_simulation.loc[usr_index,'time_lapsed'] = user_row['time_lapsed']

                    network_simulation.loc[usr_index,'source_index'] = user_row['source_index']
                    network_simulation.loc[usr_index,'seed_index'] = user_row['seed_index']
                    network_simulation.loc[usr_index, 'generation'] = user_row['generation']
                    network_simulation.loc[usr_index, 'time_since_seed'] = user_row['time_since_seed']


                network_simulation.at[usr_index,'source_candidates'] = sources

                # UsM: User metadata

                features['UsM_deltaDays0'].append(first_source_row.user_created_days)
                features['UsM_statusesCount0'].append(first_source_row.statuses_count)
                features['UsM_followersCount0'].append(first_source_row.followers_count)
                features['UsM_favouritesCount0'].append(first_source_row.favourites_count)
                features['UsM_friendsCount0'].append(first_source_row.friends_count)
                features['UsM_listedCount0'].append(first_source_row.listed_count)
                features['UsM_normalizedUserStatusesCount0'].append(first_source_row.normalized_statuses_count)
                features['UsM_normalizedUserFollowersCount0'].append(first_source_row.normalized_followers_count)
                features['UsM_normalizedUserFavouritesCount0'].append(first_source_row.normalized_favourites_count)
                features['UsM_normalizedUserListedCount0'].append(first_source_row.normalized_listed_count)
                features['UsM_normalizedUserFriendsCount0'].append(first_source_row.normalized_friends_count)
                features['UsM_deltaDays-1'].append(last_source_row.user_created_days)
                features['UsM_statusesCount-1'].append(last_source_row.statuses_count)
                features['UsM_followersCount-1'].append(last_source_row.followers_count)
                features['UsM_favouritesCount-1'].append(last_source_row.favourites_count)
                features['UsM_friendsCount-1'].append(last_source_row.friends_count)
                features['UsM_listedCount-1'].append(last_source_row.listed_count)
                features['UsM_normalizedUserStatusesCount-1'].append(last_source_row.normalized_statuses_count)
                features['UsM_normalizedUserFollowersCount-1'].append(last_source_row.normalized_followers_count)
                features['UsM_normalizedUserFavouritesCount-1'].append(last_source_row.normalized_favourites_count)
                features['UsM_normalizedUserListedCount-1'].append(last_source_row.normalized_listed_count)
                features['UsM_normalizedUserFriendsCount-1'].append(last_source_row.normalized_friends_count)
                # TwM: Tweet metadata
                features['TwM_t0'].append(round(timeList[0], 1))
                features['TwM_tSeed0'].append(round(current_time - first_source_seed_row['time_lapsed'], 1))
                features['TwM_t-1'].append(round(timeList[-1], 1))
                features['TwM_tSeed-1'].append(round(current_time - last_source_seed_row['time_lapsed'], 1))
                features['TwM_tCurrent'].append(current_time)
                # Nw: Network
                features['Nw_degree'].append(degree[index])
                features['Nw_inDegree'].append(in_degree[index])
                features['Nw_outDegree'].append(out_degree[index])
                features['Nw_degree0'].append(degree[first_source_index])
                features['Nw_inDegree0'].append(in_degree[first_source_index])
                features['Nw_outDegree0'].append(out_degree[first_source_index])
                features['Nw_degree-1'].append(degree[last_source_index])
                features['Nw_inDegree-1'].append(in_degree[last_source_index])
                features['Nw_outDegree-1'].append(out_degree[last_source_index])
                features['Nw_degreeSeed0'].append(degree[int(first_source_row['seed_index'])])
                features['Nw_inDegreeSeed0'].append(in_degree[int(first_source_row['seed_index'])])
                features['Nw_outDegreeSeed0'].append(out_degree[int(first_source_row['seed_index'])])
                features['Nw_degreeSeed-1'].append(degree[int(last_source_row['seed_index'])])
                features['Nw_inDegreeSeed-1'].append(in_degree[int(last_source_row['seed_index'])])
                features['Nw_outDegreeSeed-1'].append(out_degree[int(last_source_row['seed_index'])])
                # SNw: Spreading Network
                features['SNw_nFriendsInfected'].append(len(sources))
                features['SNw_friendsInfectedRatio'].append(safe_division(len(sources), user_row['friends_count']))
                features['SNw_generation0'].append(first_source_row['generation'])
                features['SNw_generation-1'].append(last_source_row['generation'])
                features['SNw_timeSinceSeed0'].append(first_source_row['time_since_seed'])
                features['SNw_timeSinceSeed-1'].append(last_source_row['time_since_seed'])

                infected_dataframe = network_simulation[network_simulation.time_lapsed <= current_time]
                total_nodes_infected = infected_dataframe.shape[0]
                total_in_degree = sum(infected_dataframe.friends_count)
                total_out_degree = sum(infected_dataframe.followers_count)

                features['SNw_totalNodesInfected'].append(total_nodes_infected)
                features['SNw_nodeInfectedCentrality'].append(len(sources)/total_nodes_infected)
                features['SNw_totalInDegree'].append(total_in_degree)
                features['SNw_totalOutDegree'].append(total_out_degree)
                features['SNw_inDegreeCentrality'].append(in_degree[index]/total_in_degree)
                features['SNw_inDegreeCentrality0'].append(in_degree[first_source_index]/total_in_degree)
                features['SNw_inDegreeCentrality-1'].append(in_degree[last_source_index]/total_in_degree)
                features['SNw_outDegreeCentrality'].append(out_degree[index]/total_out_degree)
                features['SNw_outDegreeCentrality0'].append(out_degree[first_source_index]/total_out_degree)
                features['SNw_outDegreeCentrality-1'].append(out_degree[last_source_index]/total_out_degree)
                features['SNw_inDegreeCentralitySeed0'].append(in_degree[int(first_source_row['seed_index'])]/total_in_degree)
                features['SNw_outDegreeCentralitySeed0'].append(out_degree[int(first_source_row['seed_index'])]/total_out_degree)
                features['SNw_inDegreeCentralitySeed-1'].append(in_degree[int(last_source_row['seed_index'])]/total_in_degree)
                features['SNw_outDegreeCentralitySeed-1'].append(out_degree[int(last_source_row['seed_index'])]/total_out_degree)
                # Stat: Statistical
                features['Stat_average_kOut'].append(round(mean(degreeList), 1))
                features['Stat_average_t'].append(round(mean(timeList), 1))
                features['Stat_average_deltaDays'].append(sources_dataframe.user_created_days.mean())
                features['Stat_average_statusesCount'].append(sources_dataframe.statuses_count.mean())
                features['Stat_average_followersCount'].append(sources_dataframe.followers_count.mean())
                features['Stat_average_favouritesCount'].append(sources_dataframe.favourites_count.mean())
                features['Stat_average_friendsCount'].append(sources_dataframe.friends_count.mean())
                features['Stat_average_listedCount'].append(sources_dataframe.listed_count.mean())
                features['Stat_average_normalizedUserStatusesCount'].append(sources_dataframe.normalized_statuses_count.mean())
                features['Stat_average_normalizedUserFollowersCount'].append(sources_dataframe.normalized_followers_count.mean())
                features['Stat_average_normalizedUserFavouritesCount'].append(sources_dataframe.normalized_favourites_count.mean())
                features['Stat_average_normalizedUserListedCount'].append(sources_dataframe.normalized_listed_count.mean())
                features['Stat_average_normalizedUserFriendsCount'].append(sources_dataframe.normalized_friends_count.mean())
                features['Stat_max_kOut'].append(max(degreeList))
                features['Stat_min_kOut'].append(min(degreeList))
            else:
                features['UsM_deltaDays0'].append(None)
                features['UsM_statusesCount0'].append(None)
                features['UsM_followersCount0'].append(None)
                features['UsM_favouritesCount0'].append(None)
                features['UsM_friendsCount0'].append(None)
                features['UsM_listedCount0'].append(None)
                features['UsM_normalizedUserStatusesCount0'].append(None)
                features['UsM_normalizedUserFollowersCount0'].append(None)
                features['UsM_normalizedUserFavouritesCount0'].append(None)
                features['UsM_normalizedUserListedCount0'].append(None)
                features['UsM_normalizedUserFriendsCount0'].append(None)
                features['UsM_deltaDays-1'].append(None)
                features['UsM_statusesCount-1'].append(None)
                features['UsM_followersCount-1'].append(None)
                features['UsM_favouritesCount-1'].append(None)
                features['UsM_friendsCount-1'].append(None)
                features['UsM_listedCount-1'].append(None)
                features['UsM_normalizedUserStatusesCount-1'].append(None)
                features['UsM_normalizedUserFollowersCount-1'].append(None)
                features['UsM_normalizedUserFavouritesCount-1'].append(None)
                features['UsM_normalizedUserListedCount-1'].append(None)
                features['UsM_normalizedUserFriendsCount-1'].append(None)
                # TwM: Tweet metadata
                features['TwM_t0'].append(None)
                features['TwM_tSeed0'].append(None)
                features['TwM_t-1'].append(None)
                features['TwM_tSeed-1'].append(None)
                features['TwM_tCurrent'].append(None)
                # Nw: Network
                features['Nw_degree'].append(None)
                features['Nw_inDegree'].append(None)
                features['Nw_outDegree'].append(None)
                features['Nw_degree0'].append(None)
                features['Nw_inDegree0'].append(None)
                features['Nw_outDegree0'].append(None)
                features['Nw_degree-1'].append(None)
                features['Nw_inDegree-1'].append(None)
                features['Nw_outDegree-1'].append(None)
                features['Nw_degreeSeed0'].append(None)
                features['Nw_inDegreeSeed0'].append(None)
                features['Nw_outDegreeSeed0'].append(None)
                features['Nw_degreeSeed-1'].append(None)
                features['Nw_inDegreeSeed-1'].append(None)
                features['Nw_outDegreeSeed-1'].append(None)
                # SNw: Spreading Network
                features['SNw_nFriendsInfected'].append(0)
                features['SNw_friendsInfectedRatio'].append(None)
                features['SNw_generation0'].append(None)
                features['SNw_generation-1'].append(None)
                features['SNw_timeSinceSeed0'].append(None)
                features['SNw_timeSinceSeed-1'].append(None)
                features['SNw_totalNodesInfected'].append(None)
                features['SNw_nodeInfectedCentrality'].append(None)
                features['SNw_totalInDegree'].append(None)
                features['SNw_totalOutDegree'].append(None)
                features['SNw_inDegreeCentrality'].append(None)
                features['SNw_inDegreeCentrality0'].append(None)
                features['SNw_inDegreeCentrality-1'].append(None)
                features['SNw_outDegreeCentrality'].append(None)
                features['SNw_outDegreeCentrality0'].append(None)
                features['SNw_outDegreeCentrality-1'].append(None)
                features['SNw_inDegreeCentralitySeed0'].append(None)
                features['SNw_outDegreeCentralitySeed0'].append(None)
                features['SNw_inDegreeCentralitySeed-1'].append(None)
                features['SNw_outDegreeCentralitySeed-1'].append(None)
                # Stat: Statistical
                features['Stat_average_kOut'].append(None)
                features['Stat_average_t'].append(None)
                features['Stat_average_deltaDays'].append(None)
                features['Stat_average_statusesCount'].append(None)
                features['Stat_average_followersCount'].append(None)
                features['Stat_average_favouritesCount'].append(None)
                features['Stat_average_friendsCount'].append(None)
                features['Stat_average_listedCount'].append(None)
                features['Stat_average_normalizedUserStatusesCount'].append(None)
                features['Stat_average_normalizedUserFollowersCount'].append(None)
                features['Stat_average_normalizedUserFavouritesCount'].append(None)
                features['Stat_average_normalizedUserListedCount'].append(None)
                features['Stat_average_normalizedUserFriendsCount'].append(None)
                features['Stat_max_kOut'].append(None)
                features['Stat_min_kOut'].append(None)

            pbar.update(1)
    processed_dataframe = pd.DataFrame(features)
    return processed_dataframe

In [34]:
ind = network_simulation.friends_count
outd = network_simulation.followers_count
out_degree = outd.tolist()
in_degree = ind.tolist()
degree = (ind+outd).tolist()

In [35]:
def run_dataset_preparation_idx():
    number_of_processes = multiprocessing.cpu_count()
    print('Will start {} processes'.format(number_of_processes))
    with Pool(number_of_processes) as pool:
        parameters = []
        number_of_users = len(network_simulation.index)
        task_size = math.ceil(number_of_users/number_of_processes)
        for i in range(number_of_processes):
            start_index = i * task_size
            end_index = min((i + 1) * task_size, number_of_users)
            parameters.append((start_index, end_index,current_time))
        dataframe_results = pool.starmap(process_data_idx, parameters)

    result = pd.DataFrame()
    result = result.append(dataframe_results)
    result.index = network_simulation.index
    save_pickle_file(path+event+'_'+str(start_hour)+"_hrs_data.pkl",result)
    print('extracted {} of rows'.format(len(result.index)))


In [36]:
run_dataset_preparation_idx()

Will start 8 processes


100%|██████████| 229057/229057 [2:42:58<00:00, 23.42it/s]   


Dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_7_hrs_data.pkl
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_7_hrs_data.pkl
extracted 1832451 of rows


In [37]:
save_pickle_file(path+event+"_network_simulation_"+str(start_hour)+"_hrs.pkl",network_simulation)

Dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_network_simulation_7_hrs.pkl
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_network_simulation_7_hrs.pkl


## Merge users and extended followers




In [8]:
input_path = "/Users/syamantak/JayateeB/dissertation/output/"

users = load_pickle_file(input_path + "nyc_users_6_9_infected.dat")
ext_followers = pd.read_csv(input_path + "nyc_6_7_ext_followers.csv")

Loading data file from /Users/syamantak/JayateeB/dissertation/output/nyc_users_6_9_infected.dat
Loaded 88266 entries


In [9]:
def id_to_index_list(idx_lookup, src_candidate_ids):
    return list(filter(lambda x: x is not None, map(lambda x: idx_lookup.get(x, None), src_candidate_ids)))


network_simulation = pd.DataFrame(columns=['id', 'time_lapsed', 'favourites_count', 'followers_count', 'friends_count',
                                           'listed_count', 'statuses_count', 'source_candidates', 'source_index',
                                           'seed_index', 'generation', 'time_since_seed', 'user_created_days',
                                           'normalized_statuses_count', 'normalized_followers_count',
                                           'normalized_favourites_count', 'normalized_listed_count',
                                           'normalized_friends_count'])

network_simulation['id']=users['id'].append(ext_followers['id'],ignore_index = True)
network_simulation['favourites_count']=users['favourites_count'].append(ext_followers['favourites_count'],ignore_index = True)
network_simulation['followers_count']=users['followers_count'].append(ext_followers['followers_count'],ignore_index = True)
network_simulation['friends_count']=users['friends_count'].append(ext_followers['friends_count'],ignore_index = True)
network_simulation['listed_count']=users['listed_count'].append(ext_followers['listed_count'],ignore_index = True)
network_simulation['statuses_count']=users['statuses_count'].append(ext_followers['statuses_count'],ignore_index = True)
network_simulation['user_created_days']=users['user_created_days'].append(ext_followers['user_created_days'],ignore_index = True)
network_simulation['normalized_statuses_count']=users['normalized_statuses_count'].append(ext_followers['normalized_statuses_count'],ignore_index = True)
network_simulation['normalized_followers_count']=users['normalized_followers_count'].append(ext_followers['normalized_followers_count'],ignore_index = True)
network_simulation['normalized_favourites_count']=users['normalized_favourites_count'].append(ext_followers['normalized_favourites_count'],ignore_index = True)
network_simulation['normalized_listed_count']=users['normalized_listed_count'].append(ext_followers['normalized_listed_count'],ignore_index = True)
network_simulation['normalized_friends_count']=users['normalized_friends_count'].append(ext_followers['normalized_friends_count'],ignore_index = True)
network_simulation['source_candidates']=users['source_candidates'].append(ext_followers['source_candidates'],ignore_index = True)
network_simulation['time_lapsed'] = users['time_lapsed'].apply(lambda x: x if x <= current_time else None)
network_simulation['source_index'] = users.apply(lambda x: x['source_index'] if x['time_lapsed'] <= current_time else None,axis=1)
network_simulation['seed_index'] = users.apply(lambda x: x['seed_index'] if x['time_lapsed'] <= current_time else None,axis=1)
network_simulation['generation'] = users.apply(lambda x: x['generation'] if x['time_lapsed'] <= current_time else None,axis=1)
network_simulation['time_since_seed'] = users.apply(lambda x: x['time_since_seed'] if x['time_lapsed'] <= current_time else None,axis=1)
network_simulation['followers_list'] = users["followers_list"]

id_list = network_simulation['id'].tolist()
index_lookup = {k: v for v, k in enumerate(network_simulation['id'].tolist())}

for i in range(88266, len(id_list)):
    source_candidate_ids = network_simulation.loc[i, "source_candidates"]
    if isinstance(source_candidate_ids, list):
        network_simulation.loc[i, "source_candidates"] = id_to_index_list(index_lookup, source_candidate_ids)


In [10]:
save_pickle_file(path+event+'_network_simulation_'+str(start_hour)+'_hrs.pkl',network_simulation)

Dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_network_simulation_7_hrs.pkl
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/nyc_network_simulation_7_hrs.pkl


In [14]:
network_simulation[network_simulation["followers_list"].isnull() == False]

,id,time_lapsed,favourites_count,followers_count,friends_count,listed_count,statuses_count,source_candidates,source_index,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count,followers_list
76100,66808635,360.02,16322,284,566,24,52913,"[2220, 9611, 4524]",2220.0,6.0,6.0,339.30,3037,17.422786,0.093513,5.374383,0.007903,0.186368,"[""1109146905110552579"", ""749694009740660736"", ..."
76101,80085947,360.02,16974,1883,1690,198,13660,"[8706, 13832, 41608, 49802, 63759, 76817, 2178...",2793.0,0.0,2.0,360.02,2989,4.570090,0.629977,5.678822,0.066243,0.565406,"[""3855128536"", ""140919963"", ""10199741879754301..."
76102,896777017,360.02,8793,1067,322,4,46600,"[283, 64074, 53907, 17094]",283.0,6.0,2.0,339.30,1877,24.826851,0.568460,4.684603,0.002131,0.171550,"[""1142871789158244354"", ""40082421"", ""786053424..."
76103,754457065,360.02,29435,582,1442,23,13779,[71517],71517.0,6.0,6.0,339.30,1947,7.077042,0.298921,15.118130,0.011813,0.740627,"[""1242386060"", ""70346818"", ""4427972598"", ""8261..."
76105,2805384220,360.05,10279,379,640,475,11391,"[76931, 4234, 29970, 59923, 51736, 49432, 1525...",283.0,6.0,2.0,339.33,1164,9.786082,0.325601,8.830756,0.408076,0.549828,"[""986617093419421697"", ""1110151699539394560"", ..."
76106,234385692,360.07,2375,1295,2916,26,2442,"[42498, 48139, 24598, 73758, 67103, 84513, 881...",996.0,6.0,5.0,339.35,2532,0.964455,0.511453,0.937994,0.010269,1.151659,"[""331859518"", ""39025928"", ""1114821470465744897..."
76107,232302233,360.08,4417,2599,1901,11,16527,"[12896, 39496, 12973, 30927, 61366, 73758]",12896.0,6.0,5.0,339.36,2538,6.511820,1.024035,1.740347,0.004334,0.749015,"[""997610240257359872"", ""1059791487116828672"", ..."
76108,2361144750,360.13,602,85,52,58,58325,"[8330, 22372]",8330.0,6.0,7.0,339.41,1385,42.111913,0.061372,0.434657,0.041877,0.037545,"[""1124823854587817984"", ""887832191740366848"", ..."
76109,57994917,360.13,41438,1879,4539,285,239216,"[2052, 56331, 11793, 9747, 6164, 39445, 75284,...",283.0,6.0,2.0,339.41,3068,77.971317,0.612451,13.506519,0.092894,1.479465,"[""1099533523436789760"", ""18539631"", ""278293693..."
76110,798236262922600448,360.13,17,322,2095,0,64,"[34818, 30724, 2054, 63494, 20492, 65548, 2049...",74.0,6.0,4.0,339.41,392,0.163265,0.821429,0.043367,0.000000,5.344388,"[""1130634601934802947"", ""1147140459871055872"",..."


In [55]:
network_simulation = final_df

In [105]:
run_dataset_preparation()
#save_pickle_file(path+event+"_network_simulation_"+str(math.trunc(current_time / 60))+"_hrs.pkl",network_simulation)


100%|██████████| 100/100 [00:01<00:00, 67.92it/s]


Dumping data to path /Users/syamantak/JayateeB/new_files/data/sample_hrs_data.pkl
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/sample_hrs_data.pkl


In [108]:
ns_dict = network_simulation.to_dict("index")
ns_dict

infected_dataframe = network_simulation[network_simulation.time_lapsed <= current_time]
total_nodes_infected = infected_dataframe.shape[0]
total_in_degree = sum(infected_dataframe.friends_count)
total_out_degree = sum(infected_dataframe.followers_count)

In [113]:
def run_dict_dataset_preparation():
#     number_of_processes = multiprocessing.cpu_count()
#     print('Will start {} processes'.format(number_of_processes))
#     with Pool(number_of_processes) as pool:
#         parameters = []
#         number_of_users = len(network_simulation.index)
#         task_size = math.ceil(number_of_users/number_of_processes)
#         for i in range(number_of_processes):
#             start_index = i * task_size
#             end_index = min((i + 1) * task_size, number_of_users)
#             parameters.append((start_index, end_index,current_time))
#         dataframe_results = pool.starmap(process_data, parameters)

#     result = pd.DataFrame()
#     result = result.append(dataframe_results)
#     #start_hour = math.trunc(current_time / 60)
#     save_pickle_file(path+event+str(start_hour)+"_hrs_data.pkl",result)
#     print('extracted {} of rows'.format(len(result.index)))

    start_index = 0
    end_index = 10000

    features = process_dict_data(start_index, end_index,current_time, ns_dict)
    save_pickle_file(path+"sample_hrs_data.pkl",features)
    
run_dict_dataset_preparation()

100%|██████████| 10000/10000 [09:42<00:00,  9.63it/s]


Dumping data to path /Users/syamantak/JayateeB/new_files/data/sample_hrs_data.pkl
Finished dumping data to path /Users/syamantak/JayateeB/new_files/data/sample_hrs_data.pkl
